In [1]:
import pandas as pd
import json
from openai import OpenAI
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import jsonlines
import shortuuid


In [2]:
import base64
import requests

# OpenAI API Key
api_key = "OPENAI_API_KEY"

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
NUM_SECONDS_TO_SLEEP = 10
def get_eval(content: str, base64_image: str, max_tokens: int):
  response = None
  while True:
    try:
      payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": content
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": max_tokens
      }
      response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
      break
    except openai.error.RateLimitError:
      pass
    except Exception as e:
      print(e)
    time.sleep(NUM_SECONDS_TO_SLEEP)
  # return response.json()['choices'][0]['message']['content']
  return response.json()['choices'][0]['message']['content']



## 이미지 전처리

In [3]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

## 메인

In [8]:
import jsonlines
from pprint import pprint
dir = '/data/MLP/cschoi/LLaVA/playground/data/eval/llava-bench-in-the-wild/questions.jsonl'
img_dir = '/data/MLP/cschoi/LLaVA/playground/data/eval/llava-bench-in-the-wild/images/'
output_dir = '/data/MLP/cschoi/LLaVA/playground/data/eval/llava-bench-in-the-wild/answers_gpt4.jsonl'
output_data = []  # List to hold the output data
# Open the JSONL file in read mode
question_id = 0
with jsonlines.open(dir, 'r') as reader:
    # Iterate over each line in the file
    for line in reader:
        # Process each line as a JSON object    
        img_path = img_dir+line['image']
        base64_image = encode_image(img_path)

        line['prompt'] = line['text']
        line['text'] = get_eval(line['text'], base64_image, 1024)
        line['model'] = 'gpt-4-vision-preview'
        line['question_id'] = question_id
        line['answer_id'] = shortuuid.uuid() 
        output_data.append(line)
        question_id += 1
        print(line)

# # Write the output data to a JSON file
with open(output_dir, 'w') as outfile:
    json.dump(output_data, outfile)
    outfile.write("\n") # json을 쓰는 것과 같지만, 여러 줄을 써주는 것이므로 "\n"을 붙여준다.

{'image': '001.jpg', 'text': 'This is Diamond Head, also known as Lēʻahi, on the island of Oʻahu in Hawaii. It is a famous volcanic tuff cone and a landmark known for its hiking trail and stunning coastal views.', 'category': 'conv', 'question_id': 0, 'prompt': 'What is the name of this famous sight in the photo?', 'model': 'gpt-4-vision-preview', 'answer_id': 'CubsN5EWqBkeUMWjLiGmSP'}
{'image': '001.jpg', 'text': "This is an aerial photograph of a striking coastal landscape. In the foreground, you see a prominent, steep-sided mountain that appears to be an extinct volcano or a volcanic crater, likely the edge of a caldera, with deep grooves eroded along its slopes. The color of the mountain is a mix of brown and rust, indicating barren or sparsely vegetated terrain.\n\nThe base of the mountain meets a stunning coastline with vivid blue ocean waters. Along the shore, there's a curved sandy beach with what looks to be a gathering of people, likely a popular spot for beachgoers, and wave